In [1]:
import pandas as pd

from src.dataset import DepressionDataset
from src.train import train
from src.model import GCN

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
raw_train = pd.read_table("./data/bronze/train.tsv", sep="\t").groupby("label").head(2000).reset_index(drop=True)
raw_dev = pd.read_table("./data/bronze/dev.tsv", sep="\t")

In [3]:
dataset_path = './data/gold'
dataset_root = dataset_path + "/w2v_window_11"

train_dataset = DepressionDataset(
    root=dataset_root, filename="",
    prefix="train",
    raw_data=raw_train
)

dev_dataset = DepressionDataset(
    root=dataset_root, filename="",
    prefix="dev",
    raw_data=raw_dev
)

In [4]:
HYPERPARAMETERS = {
    "n_epochs": [300],
    "batch_size": [32, 128, 64],
    "learning_rate": [0.1, 0.05, 0.01, 0.001],
    "weight_decay": [0.0001, 0.00001, 0.001],
    "sgd_momentum": [0.9, 0.8, 0.5],
    "scheduler_gamma": [0.995, 0.9, 0.8, 0.5, 1],
    "model_embedding_size": [8, 16, 32, 64, 128],
    "model_dense_neurons": [16, 128, 64, 256, 32],
    "model_layers": [3],
    "model_num_classes": [3],
}

In [ ]:
initial_custom

In [5]:
from mango import Tuner

config = dict()
config["optimizer"] = "Bayesian"
config["num_iteration"] = 100
congfig["initial_custom"] = initial_custom

tuner = Tuner(HYPERPARAMETERS, 
              objective=lambda params: [train(param, train_dataset, dev_dataset, GCN) for param in params],
              conf_dict=config)
results = tuner.minimize()

Ep10 | TrLoss: 0.7329 | ValLoss: 1.0788 | EarStop: 1/10:   5%|▌         | 15/300 [07:56<2:30:46, 31.74s/it]


KeyboardInterrupt: 

In [6]:
!mlflow server

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2023-11-30 09:49:11 +0100] [24862] [INFO] Starting gunicorn 21.2.0
[2023-11-30 09:49:11 +0100] [24862] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-30 09:49:11 +0100] [24862] [ERROR] Retrying in 1 second.
[2023-11-30 09:49:12 +0100] [24862] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-30 09:49:12 +0100] [24862] [ERROR] Retrying in 1 second.
[2023-11-30 09:49:13 +0100] [24862] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-30 09:49:13 +0100] [24862] [ERROR] Retry